# Lab Four: The Multi-Layer Perceptron
 

#### Everett Cienkus, Blake Miller, Colin Weil

### 1. Load, Split, and Balance

#### 1.1 Load Data

#### 1.2 Split Dataset

#### 1.3 Balance

### 2. Pre-proccesing

### 3. Modeling

### 4. Adaptive momentum (AdaM) 